In [194]:
import pandas as pd
def load_data_engeland(Y):
    d={}
    url = "https://www.football-data.co.uk/fixtures.csv"
    programma = pd.read_csv(url)
    df_programma = programma[programma['Div'] == 'E0']
    
    for i in range(0,10):
        Y1 = int(Y)-i; Y2 = int(Y)-1-i
        Y1 = str(Y1)[-2:]; Y2 = str(Y2)[-2:]
        url = 'http://www.football-data.co.uk/mmz4281/'+Y2+Y1+'/E0.csv'
        if i == 0:
            d["df{0}".format(i)]= pd.concat([pd.read_csv(url),df_programma],axis=0)
            d["df{0}".format(i)] = d["df{0}".format(i)].iloc[::-1]
        else:
            d["df{0}".format(i)]= pd.read_csv(url)
            
            # reverse, newest games on top
            d["df{0}".format(i)] = d["df{0}".format(i)].iloc[::-1]

    return d
aldata = load_data_engeland(2021)
print(aldata.keys())
#     print(i)

dict_keys(['df0', 'df1', 'df2', 'df3', 'df4', 'df5', 'df6', 'df7', 'df8', 'df9'])


In [195]:
data = load_data_engeland(2021)
data.keys()

dict_keys(['df0', 'df1', 'df2', 'df3', 'df4', 'df5', 'df6', 'df7', 'df8', 'df9'])

In [196]:
df = data

In [197]:
df['df0'].head(2)

,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,AvgC<2.5,AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA
11,E0,30/12/2020,20:00,Newcastle,Liverpool,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,E0,30/12/2020,18:00,Tottenham,Fulham,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [216]:

def create_df(df:dict) -> pd.DataFrame:
    x = list(df.keys())[0]
    index0 = list(df.keys()).index(x)
    index3 = index0 + 4
    df3 = pd.DataFrame()
    for i in range(index0,index3):
        df3 = pd.concat([df3, pd.DataFrame.from_dict(df[list(df.keys())[i]])],axis = 0,ignore_index=False).reset_index(drop=True)
    df[x] = df[x][['Date','HomeTeam','AwayTeam','FTHG','FTAG','FTR','HTHG','HTAG','HTR','B365H','B365D','B365A']].reset_index(drop=True)
    for start, row in df[x].iterrows():
#         huidig seizoen
        
        # thuisteam iterate
        t = row['HomeTeam']
        
        # uitteam iterate
        u = row['AwayTeam']

        # wedstrijden seizoen
        end = len(df[x])

        # team 1 dataframes
        df_t1 = df[x].iloc[start+1:end].loc[(df[x]['HomeTeam'] == t) | (df[x]['AwayTeam'] == t)]
        df_t1_tt = df_t1.loc[df_t1['HomeTeam'] == t]
        df_t1_ut = df_t1.loc[df_t1['AwayTeam'] == t]

        # team 2 dataframes
        df_t2 = df[x].iloc[start+1:end].loc[(df[x]['HomeTeam'] == u) | (df[x]['AwayTeam'] == u)]
        df_t2_tt = df_t2.loc[df_t2['HomeTeam'] == u]
        df_t2_ut = df_t2.loc[df_t2['AwayTeam'] == u]
        
       # team 1 punten per seizoen thuis + uit, + mean + ratio punten  
        df[x].loc[start,'t1_tt_tp'] = len(df_t1_tt[df_t1_tt['FTR'] == 'H'])*3 + len(df_t1_tt[df_t1_tt['FTR'] == 'D'])
        df[x].loc[start,'t1_ut_tp'] = len(df_t1_ut[df_t1_ut['FTR'] == 'A'])*3 + len(df_t1_ut[df_t1_ut['FTR'] == 'D'])
        df[x].loc[start,'t1_tp'] = df[x].loc[start,'t1_tt_tp'] + df[x].loc[start,'t1_ut_tp']
        
        # huidig seizoen win, gelijk en verlies
        df[x].loc[start,'t1_hs_w'] = len(df_t1_ut[df_t1_ut['FTR'] == 'A']) + len(df_t1_tt[df_t1_tt['FTR'] == 'H'])
        df[x].loc[start,'t1_hs_g'] = len(df_t1_ut[df_t1_ut['FTR'] == 'D']) + len(df_t1_tt[df_t1_tt['FTR'] == 'D'])
        df[x].loc[start,'t1_hs_v'] = len(df_t1_ut[df_t1_ut['FTR'] == 'H']) + len(df_t1_tt[df_t1_tt['FTR'] == 'A'])

        # huidig seizeon team 1 win, gelijk en verlies ratio's
        df[x].loc[start,'t1_hs_w_r'] = df[x].loc[start,'t1_hs_w'] / len(df_t1) if len(df_t1) > 0 else np.nan
        df[x].loc[start,'t1_hs_g_r'] = df[x].loc[start,'t1_hs_g'] / len(df_t1) if len(df_t1) > 0 else np.nan
        df[x].loc[start,'t1_hs_v_r'] = df[x].loc[start,'t1_hs_v'] / len(df_t1) if len(df_t1) > 0 else np.nan
    
        # huidig seizeon thuis wedstrijd team 1 win, gelijk en verlies
        df[x].loc[start,'t1_tt_hs_w_r'] = len(df_t1_tt[df_t1_tt['FTR'] == 'H']) / len(df_t1_tt) if len(df_t1_tt) != 0 else np.nan
        df[x].loc[start,'t1_tt_hs_g_r'] = len(df_t1_tt[df_t1_tt['FTR'] == 'D']) / len(df_t1_tt) if len(df_t1_tt) != 0 else np.nan
        df[x].loc[start,'t1_tt_hs_v_r'] = len(df_t1_ut[df_t1_ut['FTR'] == 'A']) / len(df_t1_tt) if len(df_t1_tt) != 0 else np.nan
        
        # huidig seizoen uit wedstrijden team 1 win, gelijk en verlies
        df[x].loc[start,'t1_ut_hs_w_r'] = len(df_t1_ut[df_t1_ut['FTR'] == 'A']) / len(df_t1_ut) if len(df_t1_ut) != 0 else np.nan
        df[x].loc[start,'t1_ut_hs_g_r'] = len(df_t1_ut[df_t1_ut['FTR'] == 'D']) / len(df_t1_ut) if len(df_t1_ut) != 0 else np.nan
        df[x].loc[start,'t1_ut_hs_v_r'] = len(df_t1_ut[df_t1_ut['FTR'] == 'H']) / len(df_t1_ut) if len(df_t1_ut) != 0 else np.nan
        
        
        
        # gemiddle punten per thuis wedstrijd
        df[x].loc[start,'t1_tt_tp_m'] = df[x].loc[start,'t1_tt_tp'] / len(df_t1_tt) if len(df_t1_tt) > 0 else 0
        
        # gemiddelde punten per uit wedstrijd
        df[x].loc[start,'t1_ut_tp_m'] = df[x].loc[start,'t1_ut_tp'] / len(df_t1_ut) if len(df_t1_ut) > 0 else 0
        
        # gemiddelde punten per wedstrijd
        df[x].loc[start,'t1_tp_m'] = df[x].loc[start,'t1_tp'] / len(df_t1) if len(df_t1) > 0 else np.nan
        
        # ratio's punten thuis/uit van totale punten
        df[x].loc[start,'t1_tt_tp_r'] = df[x].loc[start,'t1_tt_tp'] / df[x].loc[start,'t1_tp']  if df[x].loc[start,'t1_tp'] > 0 else 0
        df[x].loc[start,'t1_tt_up_r'] = df[x].loc[start,'t1_ut_tp'] / df[x].loc[start,'t1_tp']  if df[x].loc[start,'t1_tp'] > 0 else 0
        
        ##########################################       
        ################TEAM2#####################
        ##########################################
        # team 2 punten per seizoen thuis + uit, + mean + ratio punten      
        df[x].loc[start,'t2_tt_tp'] = len(df_t2_tt[df_t2_tt['FTR'] == 'H'])*3 + len(df_t2_tt[df_t2_tt['FTR'] == 'D'])
        df[x].loc[start,'t2_ut_tp'] = len(df_t2_ut[df_t2_ut['FTR'] == 'A'])*3 + len(df_t2_ut[df_t2_ut['FTR'] == 'D'])
        df[x].loc[start,'t2_tp'] = df[x].loc[start,'t2_tt_tp'] + df[x].loc[start,'t2_ut_tp']
        
        df[x].loc[start,'t2_hs_w'] = len(df_t2_ut[df_t2_ut['FTR'] == 'A']) + len(df_t2_tt[df_t2_tt['FTR'] == 'H'])
        df[x].loc[start,'t2_hs_g'] = len(df_t2_ut[df_t2_ut['FTR'] == 'D']) + len(df_t2_tt[df_t2_tt['FTR'] == 'D'])
        df[x].loc[start,'t2_hs_v'] = len(df_t2_ut[df_t2_ut['FTR'] == 'H']) + len(df_t2_tt[df_t2_tt['FTR'] == 'A'])

        df[x].loc[start,'t2_hs_w_r'] = df[x].loc[start,'t2_hs_w'] / len(df_t2) if len(df_t2) > 0 else np.nan
        df[x].loc[start,'t2_hs_g_r'] = df[x].loc[start,'t2_hs_g'] / len(df_t2) if len(df_t2) > 0 else np.nan
        df[x].loc[start,'t2_hs_v_r'] = df[x].loc[start,'t2_hs_v'] / len(df_t2) if len(df_t2) > 0 else np.nan
        
        
        # huidig seizeon thuis wedstrijd team 2 win, gelijk en verlies
        df[x].loc[start,'t2_tt_hs_w_r'] = len(df_t2_tt[df_t2_tt['FTR'] == 'H']) / len(df_t2_tt) if len(df_t2_tt) != 0 else np.nan
        df[x].loc[start,'t2_tt_hs_g_r'] = len(df_t2_tt[df_t2_tt['FTR'] == 'D']) / len(df_t2_tt) if len(df_t2_tt) != 0 else np.nan
        df[x].loc[start,'t2_tt_hs_v_r'] = len(df_t2_ut[df_t2_ut['FTR'] == 'A']) / len(df_t2_tt) if len(df_t2_tt) != 0 else np.nan
        
        # huidig seizoen uit wedstrijden team 2 win, gelijk en verlies
        df[x].loc[start,'t2_ut_hs_w_r'] = len(df_t2_ut[df_t2_ut['FTR'] == 'A']) / len(df_t2_ut) if len(df_t2_ut) != 0 else np.nan
        df[x].loc[start,'t2_ut_hs_g_r'] = len(df_t2_ut[df_t2_ut['FTR'] == 'D']) / len(df_t2_ut) if len(df_t2_ut) != 0 else np.nan
        df[x].loc[start,'t2_ut_hs_v_r'] = len(df_t2_ut[df_t2_ut['FTR'] == 'H']) / len(df_t2_ut) if len(df_t2_ut) != 0 else np.nan
        
    
        
        # gemiddeldes
        df[x].loc[start,'t2_tt_tp_m'] = df[x].loc[start,'t2_tt_tp'] / len(df_t2_tt) if len(df_t2_tt) > 0 else 0
        df[x].loc[start,'t2_ut_tp_m'] = df[x].loc[start,'t2_ut_tp'] / len(df_t2_ut) if len(df_t2_ut) > 0 else 0
        df[x].loc[start,'t2_tp_m'] = df[x].loc[start,'t2_tp'] / len(df_t2) if len(df_t2) > 0 else np.nan
        
        df[x].loc[start,'t2_tt_tp_r'] = df[x].loc[start,'t2_tt_tp'] / df[x].loc[start,'t2_tp'] if df[x].loc[start,'t2_tp'] > 0 else 0
        df[x].loc[start,'t2_tt_up_r'] = df[x].loc[start,'t2_ut_tp'] / df[x].loc[start,'t2_tp'] if df[x].loc[start,'t2_tp'] > 0 else 0
        
        #######################################
        ### laatste 3 wedstrijden variabelen###
        #######################################
        #################TEAM1#################
        if len(df_t1) >= 3:
            df_t1_3 = df_t1.iloc[0:3]
            # team 1 laatste 3 wedstrijden winst, gelijk en verlies
            df[x].loc[start,'t1_3_w'] = len(df_t1_3[(df_t1_3['FTR'] == 'H') & (df_t1_3['HomeTeam'] == t)]) + len(df_t1_3[(df_t1_3['FTR'] == 'A') & (df_t1_3['AwayTeam'] == t)])
            df[x].loc[start,'t1_3_g'] = len(df_t1_3[(df_t1_3['FTR'] == 'D') & (df_t1_3['HomeTeam'] == t)]) + len(df_t1_3[(df_t1_3['FTR'] == 'D') & (df_t1_3['AwayTeam'] == t)])
            df[x].loc[start,'t1_3_v'] = len(df_t1_3[(df_t1_3['FTR'] == 'A') & (df_t1_3['HomeTeam'] == t)]) + len(df_t1_3[(df_t1_3['FTR'] == 'H') & (df_t1_3['AwayTeam'] == t)])
            
            # team 1 laatste 3 wedstrijden winst, gelijk en verlies ratio's
            df[x].loc[start,'t1_3_w_r'] = df[x].loc[start,'t1_3_w'] / len(df_t1_3)
            df[x].loc[start,'t1_3_g_r'] = df[x].loc[start,'t1_3_g'] / len(df_t1_3)
            df[x].loc[start,'t1_3_v_r'] = df[x].loc[start,'t1_3_v'] / len(df_t1_3)
        
        # kleiner dan 3 dan wordt het een nan
        else:
            df[x].loc[start,'t1_3_w'] = np.nan 
            df[x].loc[start,'t1_3_g'] = np.nan
            df[x].loc[start,'t1_3_v'] = np.nan
            
            df[x].loc[start,'t1_3_w_r'] = np.nan
            df[x].loc[start,'t1_3_g_r'] = np.nan
            df[x].loc[start,'t1_3_v_r'] = np.nan
        
        if len(df_t1_tt) >= 3:
            df_t1_tt_3 = df_t1_tt.iloc[0:3]
            df[x].loc[start,'t1_tt_3_w'] = len(df_t1_tt_3[(df_t1_tt_3['FTR'] == 'H') & (df_t1_tt_3['HomeTeam'] == t)]) + len(df_t1_tt_3[(df_t1_tt_3['FTR'] == 'A') & (df_t1_tt_3['AwayTeam'] == t)])
            df[x].loc[start,'t1_tt_3_g'] = len(df_t1_tt_3[(df_t1_tt_3['FTR'] == 'D') & (df_t1_tt_3['HomeTeam'] == t)]) + len(df_t1_tt_3[(df_t1_tt_3['FTR'] == 'D') & (df_t1_tt_3['AwayTeam'] == t)])
            df[x].loc[start,'t1_tt_3_v'] = len(df_t1_tt_3[(df_t1_tt_3['FTR'] == 'A') & (df_t1_tt_3['HomeTeam'] == t)]) + len(df_t1_tt_3[(df_t1_tt_3['FTR'] == 'H') & (df_t1_tt_3['AwayTeam'] == t)])
            
            df[x].loc[start,'t1_tt_3_w_r'] = df[x].loc[start,'t1_tt_3_w'] / len(df_t1_tt_3)
            df[x].loc[start,'t1_tt_3_g_r'] = df[x].loc[start,'t1_tt_3_g'] / len(df_t1_tt_3)
            df[x].loc[start,'t1_tt_3_v_r'] = df[x].loc[start,'t1_tt_3_v'] / len(df_t1_tt_3)
            
        else:
            df[x].loc[start,'t1_tt_3_w'] = np.nan 
            df[x].loc[start,'t1_tt_3_g'] = np.nan
            df[x].loc[start,'t1_tt_3_v'] = np.nan
            
            df[x].loc[start,'t1_tt_3_w_r'] = np.nan
            df[x].loc[start,'t1_tt_3_g_r'] = np.nan
            df[x].loc[start,'t1_tt_3_v_r'] = np.nan
        
        if len(df_t1_ut) >= 3:
            df_t1_ut_3 = df_t1_ut.iloc[0:3]
            df[x].loc[start,'t1_ut_3_w'] = len(df_t1_ut_3[(df_t1_ut_3['FTR'] == 'H') & (df_t1_ut_3['HomeTeam'] == t)]) + len(df_t1_ut_3[(df_t1_ut_3['FTR'] == 'A') & (df_t1_ut_3['AwayTeam'] == t)])
            df[x].loc[start,'t1_ut_3_g'] = len(df_t1_ut_3[(df_t1_ut_3['FTR'] == 'D') & (df_t1_ut_3['HomeTeam'] == t)]) + len(df_t1_ut_3[(df_t1_ut_3['FTR'] == 'D') & (df_t1_ut_3['AwayTeam'] == t)])
            df[x].loc[start,'t1_ut_3_v'] = len(df_t1_ut_3[(df_t1_ut_3['FTR'] == 'A') & (df_t1_ut_3['HomeTeam'] == t)]) + len(df_t1_ut_3[(df_t1_ut_3['FTR'] == 'H') & (df_t1_ut_3['AwayTeam'] == t)])
            
            df[x].loc[start,'t1_ut_3_w_r'] = df[x].loc[start,'t1_ut_3_w'] / len(df_t1_ut_3)
            df[x].loc[start,'t1_ut_3_g_r'] = df[x].loc[start,'t1_ut_3_g'] / len(df_t1_ut_3)
            df[x].loc[start,'t1_ut_3_v_r'] = df[x].loc[start,'t1_ut_3_v'] / len(df_t1_ut_3)
            
        else:
            df[x].loc[start,'t1_ut_3_w'] = np.nan 
            df[x].loc[start,'t1_ut_3_g'] = np.nan
            df[x].loc[start,'t1_ut_3_v'] = np.nan
            
            df[x].loc[start,'t1_ut_3_w_r'] = np.nan
            df[x].loc[start,'t1_ut_3_g_r'] = np.nan
            df[x].loc[start,'t1_ut_3_v_r'] = np.nan
        
        #######################################
        ### laatste 3 wedstrijden variabelen###
        #######################################
        #################TEAM2#################
        if len(df_t2) >= 3:
            df_t2_3 = df_t2.iloc[0:3]

            df[x].loc[start,'t2_3_w'] = len(df_t2_3[(df_t2_3['FTR'] == 'H') & (df_t2_3['HomeTeam'] == u)]) + len(df_t2_3[(df_t2_3['FTR'] == 'A') & (df_t2_3['AwayTeam'] == u)])
            df[x].loc[start,'t2_3_g'] = len(df_t2_3[(df_t2_3['FTR'] == 'D') & (df_t2_3['HomeTeam'] == u)]) + len(df_t2_3[(df_t2_3['FTR'] == 'D') & (df_t2_3['AwayTeam'] == u)])
            df[x].loc[start,'t2_3_v'] = len(df_t2_3[(df_t2_3['FTR'] == 'A') & (df_t2_3['HomeTeam'] == u)]) + len(df_t2_3[(df_t2_3['FTR'] == 'H') & (df_t2_3['AwayTeam'] == u)])
            
            df[x].loc[start,'t2_3_w_r'] = df[x].loc[start,'t2_3_w'] / len(df_t2_3)
            df[x].loc[start,'t2_3_g_r'] = df[x].loc[start,'t2_3_g'] / len(df_t2_3)
            df[x].loc[start,'t2_3_v_r'] = df[x].loc[start,'t2_3_v'] / len(df_t2_3)
            
        else:
            df[x].loc[start,'t2_3_w'] = np.nan 
            df[x].loc[start,'t2_3_g'] = np.nan
            df[x].loc[start,'t2_3_v'] = np.nan
            
            df[x].loc[start,'t2_3_w_r'] = np.nan
            df[x].loc[start,'t2_3_g_r'] = np.nan
            df[x].loc[start,'t2_3_v_r'] = np.nan
        
        if len(df_t2_tt) >= 3:
            df_t2_tt_3 = df_t2_tt.iloc[0:3]
            df[x].loc[start,'t2_tt_3_w'] = len(df_t2_tt_3[(df_t2_tt_3['FTR'] == 'H') & (df_t2_tt_3['HomeTeam'] == u)]) + len(df_t2_tt_3[(df_t2_tt_3['FTR'] == 'A') & (df_t2_tt_3['AwayTeam'] == u)])
            df[x].loc[start,'t2_tt_3_g'] = len(df_t2_tt_3[(df_t2_tt_3['FTR'] == 'D') & (df_t2_tt_3['HomeTeam'] == u)]) + len(df_t2_tt_3[(df_t2_tt_3['FTR'] == 'D') & (df_t2_tt_3['AwayTeam'] == u)])
            df[x].loc[start,'t2_tt_3_v'] = len(df_t2_tt_3[(df_t2_tt_3['FTR'] == 'A') & (df_t2_tt_3['HomeTeam'] == u)]) + len(df_t2_tt_3[(df_t2_tt_3['FTR'] == 'H') & (df_t2_tt_3['AwayTeam'] == u)])
            
            df[x].loc[start,'t2_tt_3_w_r'] = df[x].loc[start,'t2_tt_3_w'] / len(df_t2_tt_3)
            df[x].loc[start,'t2_tt_3_g_r'] = df[x].loc[start,'t2_tt_3_g'] / len(df_t2_tt_3)
            df[x].loc[start,'t2_tt_3_v_r'] = df[x].loc[start,'t2_tt_3_v'] / len(df_t2_tt_3)
            
        else:
            df[x].loc[start,'t2_tt_3_w'] = np.nan 
            df[x].loc[start,'t2_tt_3_g'] = np.nan
            df[x].loc[start,'t2_tt_3_v'] = np.nan
            
            df[x].loc[start,'t2_tt_3_w_r'] = np.nan
            df[x].loc[start,'t2_tt_3_g_r'] = np.nan
            df[x].loc[start,'t2_tt_3_v_r'] = np.nan
        
        if len(df_t2_ut) >= 3:
            df_t2_ut_3 = df_t2_ut.iloc[0:3]
            df[x].loc[start,'t2_ut_3_w'] = len(df_t2_ut_3[(df_t2_ut_3['FTR'] == 'H') & (df_t2_ut_3['HomeTeam'] == u)]) + len(df_t2_ut_3[(df_t2_ut_3['FTR'] == 'A') & (df_t2_ut_3['AwayTeam'] == u)])
            df[x].loc[start,'t2_ut_3_g'] = len(df_t2_ut_3[(df_t2_ut_3['FTR'] == 'D') & (df_t2_ut_3['HomeTeam'] == u)]) + len(df_t2_ut_3[(df_t2_ut_3['FTR'] == 'D') & (df_t2_ut_3['AwayTeam'] == u)])
            df[x].loc[start,'t2_ut_3_v'] = len(df_t2_ut_3[(df_t2_ut_3['FTR'] == 'A') & (df_t2_ut_3['HomeTeam'] == u)]) + len(df_t2_ut_3[(df_t2_ut_3['FTR'] == 'H') & (df_t2_ut_3['AwayTeam'] == u)])
            
            df[x].loc[start,'t2_ut_3_w_r'] = df[x].loc[start,'t2_ut_3_w'] / len(df_t2_ut_3)
            df[x].loc[start,'t2_ut_3_g_r'] = df[x].loc[start,'t2_ut_3_g'] / len(df_t2_ut_3)
            df[x].loc[start,'t2_ut_3_v_r'] = df[x].loc[start,'t2_ut_3_v'] / len(df_t2_ut_3)
            
        else:
            
            df[x].loc[start,'t2_ut_3_w'] = np.nan 
            df[x].loc[start,'t2_ut_3_g'] = np.nan
            df[x].loc[start,'t2_ut_3_v'] = np.nan
            
            df[x].loc[start,'t2_ut_3_w_r'] = np.nan
            df[x].loc[start,'t2_ut_3_g_r'] = np.nan
            df[x].loc[start,'t2_ut_3_v_r'] = np.nan        
            
            
            

        ###########################################################
        #########################3YEARS############################
        ###########################################################
        
        # team 1 dataframes
        df_t1_3y = df3.loc[(df3['HomeTeam'] == t) | (df3['AwayTeam'] == t)].iloc[1:]
        df_t1_tt_3y = df_t1_3y.loc[df_t1_3y['HomeTeam'] == t]
        df_t1_ut_3y = df_t1_3y.loc[df_t1_3y['AwayTeam'] == t]
        
        # team 2 dataframes
        df_t2_3y = df3.loc[(df3['HomeTeam'] == u) | (df3['AwayTeam'] == u)].iloc[1:]
        df_t2_tt_3y = df_t2_3y.loc[df_t2_3y['HomeTeam'] == u]
        df_t2_ut_3y = df_t2_3y.loc[df_t2_3y['AwayTeam'] == u]
        
        # team 1 counts win, gelijk en verlies
        df[x].loc[start,'t1_tt_w_3y'] = len(df_t1_tt_3y[df_t1_tt_3y['FTR'] == 'H'])
        df[x].loc[start,'t1_ut_w_3y'] = len(df_t1_ut_3y[df_t1_ut_3y['FTR'] == 'A'])
        df[x].loc[start,'t1_w_3y'] = len(df_t1_tt_3y[df_t1_tt_3y['FTR'] == 'H']) + len(df_t1_ut_3y[df_t1_ut_3y['FTR'] == 'A'])
        
        # team 1 thuiswedstrijden laatste 3 seizoenen thuis win, gelijk en verlies
        df[x].loc[start,'t1_tt_w_r_3y'] = len(df_t1_tt_3y[df_t1_tt_3y['FTR'] == 'H']) / len(df_t1_tt_3y) if len(df_t1_tt_3y) > 0 else np.nan
        df[x].loc[start,'t1_tt_g_r_3y'] = len(df_t1_tt_3y[df_t1_tt_3y['FTR'] == 'D']) / len(df_t1_tt_3y) if len(df_t1_tt_3y) > 0 else np.nan
        df[x].loc[start,'t1_tt_v_r_3y'] = len(df_t1_tt_3y[df_t1_tt_3y['FTR'] == 'A']) / len(df_t1_tt_3y) if len(df_t1_tt_3y) > 0 else np.nan
        
        # team 1 uitwedstrijden laatste 3 seizoenen thuis win, gelijk en verlies
        df[x].loc[start,'t1_ut_w_r_3y'] = len(df_t1_ut_3y[df_t1_ut_3y['FTR'] == 'A']) / len(df_t1_ut_3y) if len(df_t1_ut_3y) > 0 else np.nan
        df[x].loc[start,'t1_ut_g_r_3y'] = len(df_t1_ut_3y[df_t1_ut_3y['FTR'] == 'D']) / len(df_t1_ut_3y) if len(df_t1_ut_3y) > 0 else np.nan
        df[x].loc[start,'t1_ut_v_r_3y'] = len(df_t1_ut_3y[df_t1_ut_3y['FTR'] == 'H']) / len(df_t1_ut_3y) if len(df_t1_ut_3y) > 0 else np.nan
        
        # team 1 wedstirjden laatste 3 seizoenen thuis win, gelijk en verlies
        df[x].loc[start,'t1_w_r_3y'] = (len(df_t1_tt_3y[df_t1_tt_3y['FTR'] == 'H']) + len(df_t1_ut_3y[df_t1_ut_3y['FTR'] == 'A'])) / len(df_t1_3y) if len(df_t1_3y) != 0 else np.nan
        df[x].loc[start,'t1_g_r_3y'] = (len(df_t1_tt_3y[df_t1_tt_3y['FTR'] == 'D']) + len(df_t1_ut_3y[df_t1_ut_3y['FTR'] == 'D'])) / len(df_t1_3y) if len(df_t1_3y) != 0 else np.nan
        df[x].loc[start,'t1_v_r_3y'] = (len(df_t1_tt_3y[df_t1_tt_3y['FTR'] == 'A']) + len(df_t1_ut_3y[df_t1_ut_3y['FTR'] == 'H'])) / len(df_t1_3y) if len(df_t1_3y) != 0 else np.nan
        
        
        # team 2 thuiswedstrijden laatste 3 seizoenen thuis win, gelijk en verlies
        df[x].loc[start,'t2_tt_w_r_3y'] = len(df_t2_tt_3y[df_t2_tt_3y['FTR'] == 'H']) / len(df_t2_tt_3y) if len(df_t2_tt_3y) > 0 else np.nan
        df[x].loc[start,'t2_tt_g_r_3y'] = len(df_t2_tt_3y[df_t2_tt_3y['FTR'] == 'D']) / len(df_t2_tt_3y) if len(df_t2_tt_3y) > 0 else np.nan
        df[x].loc[start,'t2_tt_v_r_3y'] = len(df_t2_tt_3y[df_t2_tt_3y['FTR'] == 'A']) / len(df_t2_tt_3y) if len(df_t2_tt_3y) > 0 else np.nan
        
        # team 2 uitwedstrijden laatste 3 seizoenen thuis win, gelijk en verlies
        df[x].loc[start,'t2_ut_w_r_3y'] = len(df_t2_ut_3y[df_t2_ut_3y['FTR'] == 'A']) / len(df_t2_ut_3y) if len(df_t2_ut_3y) > 0 else np.nan
        df[x].loc[start,'t2_ut_g_r_3y'] = len(df_t2_ut_3y[df_t2_ut_3y['FTR'] == 'D']) / len(df_t2_ut_3y) if len(df_t2_ut_3y) > 0 else np.nan
        df[x].loc[start,'t2_ut_v_r_3y'] = len(df_t2_ut_3y[df_t2_ut_3y['FTR'] == 'H']) / len(df_t2_ut_3y) if len(df_t2_ut_3y) > 0 else np.nan
        
        # team 2 wedstirjden laatste 3 seizoenen thuis win, gelijk en verlies
        df[x].loc[start,'t2_w_r_3y'] = (len(df_t2_tt_3y[df_t2_tt_3y['FTR'] == 'H']) + len(df_t2_ut_3y[df_t2_ut_3y['FTR'] == 'A'])) / len(df_t2_3y) if len(df_t2_3y) != 0 else np.nan
        df[x].loc[start,'t2_g_r_3y'] = (len(df_t2_tt_3y[df_t2_tt_3y['FTR'] == 'D']) + len(df_t2_ut_3y[df_t2_ut_3y['FTR'] == 'D'])) / len(df_t2_3y) if len(df_t2_3y) != 0 else np.nan
        df[x].loc[start,'t2_v_r_3y'] = (len(df_t2_tt_3y[df_t2_tt_3y['FTR'] == 'A']) + len(df_t2_ut_3y[df_t2_ut_3y['FTR'] == 'H'])) / len(df_t2_3y) if len(df_t2_3y) != 0 else np.nan
        
        # gemeenschappelijk dataframe team 1 en team 2
        df_t1_t2_3y = df3.loc[((df3['HomeTeam'] == t) | (df3['HomeTeam'] == u)) & ((df3['AwayTeam'] == t) | (df3['AwayTeam'] == u))].iloc[1:]

        df[x].loc[start,'df_t1_t2_t1w_r'] = (len(df_t1_t2_3y[(df_t1_t2_3y['HomeTeam'] == t) & (df_t1_t2_3y['FTR'] == 'H')]) + len(df_t1_t2_3y[(df_t1_t2_3y['AwayTeam'] == t) & (df_t1_t2_3y['FTR'] == 'A')])) / len(df_t1_t2_3y) if len(df_t1_t2_3y) != 0 else np.nan
        df[x].loc[start,'df_t1_t2_t1v_r'] = (len(df_t1_t2_3y[(df_t1_t2_3y['HomeTeam'] == u) & (df_t1_t2_3y['FTR'] == 'H')]) + len(df_t1_t2_3y[(df_t1_t2_3y['AwayTeam'] == u) & (df_t1_t2_3y['FTR'] == 'A')])) / len(df_t1_t2_3y) if len(df_t1_t2_3y) != 0 else np.nan
        df[x].loc[start,'df_t1_t2_g_r'] = (len(df_t1_t2_3y[(df_t1_t2_3y['HomeTeam'] == t) & (df_t1_t2_3y['FTR'] == 'D')])+ len(df_t1_t2_3y[(df_t1_t2_3y['AwayTeam'] == t) & (df_t1_t2_3y['FTR'] == 'D')])) / len(df_t1_t2_3y) if len(df_t1_t2_3y) != 0 else np.nan
        
        
    return df[x]

In [199]:
dft.head()

""


In [201]:
d = ({k:aldata[k] for k in list(aldata.keys())[0:4]})

In [217]:
create_df(d)

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,B365H,...,t2_tt_v_r_3y,t2_ut_w_r_3y,t2_ut_g_r_3y,t2_ut_v_r_3y,t2_w_r_3y,t2_g_r_3y,t2_v_r_3y,df_t1_t2_t1w_r,df_t1_t2_t1v_r,df_t1_t2_g_r
0,30/12/2020,Newcastle,Liverpool,NaN,NaN,NaN,NaN,NaN,NaN,12.00,...,0.000000,0.584615,0.261538,0.153846,0.707692,0.215385,0.076923,0.000000,0.714286,0.285714
1,30/12/2020,Tottenham,Fulham,NaN,NaN,NaN,NaN,NaN,NaN,1.50,...,0.518519,0.076923,0.153846,0.769231,0.169811,0.188679,0.641509,1.000000,0.000000,0.000000
2,29/12/2020,Man United,Wolves,NaN,NaN,NaN,NaN,NaN,NaN,1.53,...,0.239130,0.347826,0.260870,0.391304,0.402174,0.282609,0.315217,0.200000,0.200000,0.600000
3,29/12/2020,West Brom,Leeds,NaN,NaN,NaN,NaN,NaN,NaN,3.60,...,0.375000,0.500000,0.000000,0.500000,0.437500,0.125000,0.437500,0.000000,1.000000,0.000000
4,29/12/2020,Southampton,West Ham,NaN,NaN,NaN,NaN,NaN,NaN,2.10,...,0.353846,0.246154,0.246154,0.507692,0.315385,0.253846,0.430769,0.142857,0.714286,0.142857
5,29/12/2020,Burnley,Sheffield United,NaN,NaN,NaN,NaN,NaN,NaN,2.37,...,0.481481,0.148148,0.370370,0.481481,0.259259,0.259259,0.481481,0.333333,0.333333,0.333333
6,29/12/2020,Brighton,Arsenal,NaN,NaN,NaN,NaN,NaN,NaN,3.25,...,0.169231,0.276923,0.261538,0.461538,0.461538,0.223077,0.315385,0.428571,0.285714,0.285714
7,28/12/2020,Everton,Man City,NaN,NaN,NaN,NaN,NaN,NaN,6.50,...,0.078125,0.687500,0.125000,0.187500,0.757812,0.109375,0.132812,0.000000,0.833333,0.166667
8,28/12/2020,Chelsea,Aston Villa,NaN,NaN,NaN,NaN,NaN,NaN,1.61,...,0.461538,0.269231,0.230769,0.500000,0.326923,0.192308,0.480769,0.666667,0.000000,0.333333
9,28/12/2020,Crystal Palace,Leicester,NaN,NaN,NaN,NaN,NaN,NaN,3.75,...,0.338462,0.384615,0.215385,0.400000,0.415385,0.215385,0.369231,0.571429,0.285714,0.142857


In [218]:
# concat all
df_test = pd.concat([create_df({k:aldata[k] for k in list(aldata.keys())[x:x+5]}) for x in range(0,4)]).reset_index(drop=True)

In [221]:
df_test.shape

(1305, 112)

In [220]:
df_test.to_pickle("test_football.pkl")

In [ ]:
# d3 = {
#     "t1_wO3y":[],
#     "t1_gO3y":[],
#     "t1_vO3y":[],
#     "t1_wO3yr":[],
#     "t1_gO3yr":[],
#     "t1_vO3yr":[],
    
#     "t1_ttwO3y":[],
#     "t1_ttgO3y":[],
#     "t1_ttvO3y":[],
#     "t1_ttwO3yr":[],
#     "t1_ttgO3yr":[],
#     "t1_ttvO3yr":[],
    
#     "t1_utwO3y":[],
#     "t1_utgO3y":[],
#     "t1_utvO3y":[],
#     "t1_utwO3yr":[],
#     "t1_utgO3yr":[],
#     "t1_utvO3yr":[],
#     "t1_ttgfO3yr":[],
#     "t1_ttgaO3yr":[],
#     "t1_utgfO3yr":[],
#     "t1_utgaO3yr":[],

#     "t1_w3y":[],
#     "t1_g3y":[],
#     "t1_v3y":[],
#     "t1_w3yr":[],
#     "t1_g3yr":[],
#     "t1_v3yr":[],
#     "t1_tt_w3y":[],
#     "t1_tt_g3y":[],
#     "t1_tt_v3y":[],
#     "t1_tt_w3yr":[],
#     "t1_tt_g3yr":[],
#     "t1_tt_v3yr":[],
#     "t1_tt_ga3y":[],
#     "t1_tt_gf3y":[],
#     "t1_tt_tg3y":[],
#     "t1_ut_w3y":[],
#     "t1_ut_g3y":[],
#     "t1_ut_v3y":[],
#     "t1_ut_w3yr":[],
#     "t1_ut_g3yr":[],
#     "t1_ut_v3yr":[],
#     "t1_ut_ga3y":[],
#     "t1_ut_gf3y":[],
#     "t1_ut_tg3y":[],
#     "t1_gf3y":[],
#     't1_ga3y':[],
#     "t1_tg3y":[],
#     "t2_w3y":[],
#     "t2_g3y":[],
#     "t2_v3y":[],
#     "t2_w3yr":[],
#     "t2_g3yr":[],
#     "t2_v3yr":[],
#     "t2_tt_w3y":[],
#     "t2_tt_g3y":[],
#     "t2_tt_v3y":[],
#     "t2_tt_w3yr":[],
#     "t2_tt_g3yr":[],
#     "t2_tt_v3yr":[],
#     "t2_tt_ga3y":[],
#     "t2_tt_gf3y":[],
#     "t2_tt_tg3y":[],
#     "t2_ut_w3y":[],
#     "t2_ut_g3y":[],
#     "t2_ut_v3y":[],
#     "t2_ut_w3yr":[],
#     "t2_ut_g3yr":[],
#     "t2_ut_v3yr":[],
#     "t2_ut_ga3y":[],
#     "t2_ut_gf3y":[],
#     "t2_ut_tg3y":[],
#     "t2_gf3y":[],
#     't2_ga3y':[],
#     "t2_tg3y":[]

# }


#     for start, (t,u) in enumerate((zip(df3['HomeTeam'],df3['AwayTeam']))):

#         df_t1t2_3y = df3[((df3['HomeTeam'] == t) | (df3['AwayTeam'] == t)) & ((df3['HomeTeam'] == u) | (df3['AwayTeam'] == u))]
#         df_t1ttt2ut_3y = df3[(df3['HomeTeam'] == t) & (df3['AwayTeam'] == u)]
#         df_t1utt2tt_3y = df3[(df3['HomeTeam'] == u) & (df3['AwayTeam'] == t)]


#         df_t1_3y = df3[(df3['HomeTeam'] == t) | (df3['AwayTeam'] == t)]
#         df_t1_tt_3y = df3[(df3['HomeTeam'] == t)]
#         df_t1_ut_3y = df3[(df3['AwayTeam'] == t)]

#         df_t2_3y = df3[(df3['HomeTeam'] == u) | (df3['AwayTeam'] == u)]
#         df_t2_tt_3y = df3[(df3['HomeTeam'] == u)]
#         df_t2_ut_3y = df3[(df3['AwayTeam'] == u)]

#             #T1 and T2 ontmoeting totaal
#         if len(df_t1t2_3y) != 0:
#             d3["t1_wO3y"].append(len(df_t1t2_3y[((df_t1t2_3y['HomeTeam'] == t) & (df_t1t2_3y['FTR'] =='H')) | ((df_t1t2_3y['AwayTeam'] == t) & (df_t1t2_3y['FTR'] =='A'))]))
#             d3["t1_gO3y"].append((len(df_t1t2_3y[((df_t1t2_3y['HomeTeam'] == t) & (df_t1t2_3y['FTR'] =='D')) | ((df_t1t2_3y['AwayTeam'] == t) & (df_t1t2_3y['FTR'] =='D'))])))
#             d3["t1_vO3y"].append(len(df_t1t2_3y[((df_t1t2_3y['HomeTeam'] == t) & (df_t1t2_3y['FTR'] =='D')) | ((df_t1t2_3y['AwayTeam'] == t) & (df_t1t2_3y['FTR'] =='H'))]))
#             d3["t1_wO3yr"].append((len(df_t1t2_3y[((df_t1t2_3y['HomeTeam'] == t) & (df_t1t2_3y['FTR'] =='H')) | ((df_t1t2_3y['AwayTeam'] == t) & (df_t1t2_3y['FTR'] =='A'))]))/len(df_t1t2_3y))
#             d3["t1_gO3yr"].append((len(df_t1t2_3y[((df_t1t2_3y['HomeTeam'] == t) & (df_t1t2_3y['FTR'] =='D')) | ((df_t1t2_3y['AwayTeam'] == t) & (df_t1t2_3y['FTR'] =='D'))]))/len(df_t1t2_3y))
#             d3["t1_vO3yr"].append((len(df_t1t2_3y[((df_t1t2_3y['HomeTeam'] == t) & (df_t1t2_3y['FTR'] =='A')) | ((df_t1t2_3y['AwayTeam'] == t) & (df_t1t2_3y['FTR'] =='H'))]))/len(df_t1t2_3y))
#         else:
#             d3["t1_wO3y"].append(np.nan)
#             d3["t1_gO3y"].append(np.nan)
#             d3["t1_vO3y"].append(np.nan)
#             d3["t1_wO3yr"].append(np.nan)
#             d3["t1_gO3yr"].append(np.nan)
#             d3["t1_vO3yr"].append(np.nan)

#             # t1 tt t2ut
#         if len(df_t1ttt2ut_3y) != 0:
#             d3["t1_ttwO3y"].append(len(df_t1ttt2ut_3y[((df_t1ttt2ut_3y['HomeTeam'] == t) & (df_t1ttt2ut_3y['FTR'] =='H')) | ((df_t1ttt2ut_3y['AwayTeam'] == t) & (df_t1ttt2ut_3y['FTR'] =='A'))]))
#             d3["t1_ttgO3y"].append((len(df_t1ttt2ut_3y[((df_t1ttt2ut_3y['HomeTeam'] == t) & (df_t1ttt2ut_3y['FTR'] =='D')) | ((df_t1ttt2ut_3y['AwayTeam'] == t) & (df_t1ttt2ut_3y['FTR'] =='D'))])))
#             d3["t1_ttvO3y"].append(len(df_t1ttt2ut_3y[((df_t1ttt2ut_3y['HomeTeam'] == t) & (df_t1ttt2ut_3y['FTR'] =='D')) | ((df_t1ttt2ut_3y['AwayTeam'] == t) & (df_t1ttt2ut_3y['FTR'] =='H'))]))
#             d3["t1_ttwO3yr"].append((len(df_t1ttt2ut_3y[((df_t1ttt2ut_3y['HomeTeam'] == t) & (df_t1ttt2ut_3y['FTR'] =='H')) | ((df_t1ttt2ut_3y['AwayTeam'] == t) & (df_t1ttt2ut_3y['FTR'] =='A'))]))/len(df_t1ttt2ut_3y))
#             d3["t1_ttgO3yr"].append((len(df_t1ttt2ut_3y[((df_t1ttt2ut_3y['HomeTeam'] == t) & (df_t1ttt2ut_3y['FTR'] =='D')) | ((df_t1ttt2ut_3y['AwayTeam'] == t) & (df_t1ttt2ut_3y['FTR'] =='D'))]))/len(df_t1ttt2ut_3y))
#             d3["t1_ttvO3yr"].append((len(df_t1ttt2ut_3y[((df_t1ttt2ut_3y['HomeTeam'] == t) & (df_t1ttt2ut_3y['FTR'] =='A')) | ((df_t1ttt2ut_3y['AwayTeam'] == t) & (df_t1ttt2ut_3y['FTR'] =='H'))]))/len(df_t1ttt2ut_3y))

#             d3["t1_ttgfO3yr"].append(sum(df_t1ttt2ut_3y['FTHG'])/len(df_t1ttt2ut_3y))
#             d3["t1_ttgaO3yr"].append(sum(df_t1ttt2ut_3y['FTAG'])/len(df_t1ttt2ut_3y))


#         else:
#             d3["t1_ttwO3y"].append(np.nan)
#             d3["t1_ttgO3y"].append(np.nan)
#             d3["t1_ttvO3y"].append(np.nan)
#             d3["t1_ttwO3yr"].append(np.nan)
#             d3["t1_ttgO3yr"].append(np.nan)
#             d3["t1_ttvO3yr"].append(np.nan)
#             d3["t1_ttgfO3yr"].append(np.nan)
#             d3["t1_ttgaO3yr"].append(np.nan)

#             # t1 utteam t2 ut ontmoeting
#         if len(df_t1utt2tt_3y) != 0:
#             d3["t1_utwO3y"].append(len(df_t1utt2tt_3y[((df_t1utt2tt_3y['HomeTeam'] == t) & (df_t1utt2tt_3y['FTR'] =='H')) | ((df_t1utt2tt_3y['AwayTeam'] == t) & (df_t1utt2tt_3y['FTR'] =='A'))]))
#             d3["t1_utgO3y"].append((len(df_t1utt2tt_3y[((df_t1utt2tt_3y['HomeTeam'] == t) & (df_t1utt2tt_3y['FTR'] =='D')) | ((df_t1utt2tt_3y['AwayTeam'] == t) & (df_t1utt2tt_3y['FTR'] =='D'))])))
#             d3["t1_utvO3y"].append(len(df_t1utt2tt_3y[((df_t1utt2tt_3y['HomeTeam'] == t) & (df_t1utt2tt_3y['FTR'] =='A')) | ((df_t1utt2tt_3y['AwayTeam'] == t) & (df_t1utt2tt_3y['FTR'] =='H'))]))
#             d3["t1_utwO3yr"].append((len(df_t1utt2tt_3y[((df_t1utt2tt_3y['HomeTeam'] == t) & (df_t1utt2tt_3y['FTR'] =='H')) | ((df_t1utt2tt_3y['AwayTeam'] == t) & (df_t1utt2tt_3y['FTR'] =='A'))]))/len(df_t1utt2tt_3y))
#             d3["t1_utgO3yr"].append((len(df_t1utt2tt_3y[((df_t1utt2tt_3y['HomeTeam'] == t) & (df_t1utt2tt_3y['FTR'] =='D')) | ((df_t1utt2tt_3y['AwayTeam'] == t) & (df_t1utt2tt_3y['FTR'] =='D'))]))/len(df_t1utt2tt_3y))
#             d3["t1_utvO3yr"].append((len(df_t1utt2tt_3y[((df_t1utt2tt_3y['HomeTeam'] == t) & (df_t1utt2tt_3y['FTR'] =='A')) | ((df_t1utt2tt_3y['AwayTeam'] == t) & (df_t1utt2tt_3y['FTR'] =='H'))]))/len(df_t1utt2tt_3y))
#             d3["t1_utgaO3yr"].append(sum(df_t1utt2tt_3y['FTHG'])/len(df_t1utt2tt_3y))
#             d3["t1_utgfO3yr"].append(sum(df_t1utt2tt_3y['FTAG'])/len(df_t1utt2tt_3y))

#         else:
#             d3["t1_utwO3y"].append(np.nan)
#             d3["t1_utgO3y"].append(np.nan)
#             d3["t1_utvO3y"].append(np.nan)
#             d3["t1_utwO3yr"].append(np.nan)
#             d3["t1_utgO3yr"].append(np.nan)
#             d3["t1_utvO3yr"].append(np.nan) 
#             d3["t1_utgfO3yr"].append(np.nan)
#             d3["t1_utgaO3yr"].append(np.nan)


#         if len(df_t1_3y) > 0:
#             d3["t1_w3y"].append(len(df_t1_3y[((df_t1_3y['HomeTeam'] == t) & (df_t1_3y['FTR'] =='H')) | ((df_t1_3y['AwayTeam'] == t) & (df_t1_3y['FTR'] =='A'))]))
#             d3["t1_g3y"].append((len(df_t1_3y[((df_t1_3y['HomeTeam'] == t) & (df_t1_3y['FTR'] =='D')) | ((df_t1_3y['AwayTeam'] == t) & (df_t1_3y['FTR'] =='D'))])))
#             d3["t1_v3y"].append(len(df_t1_3y[((df_t1_3y['HomeTeam'] == t) & (df_t1_3y['FTR'] =='A')) | ((df_t1_3y['AwayTeam'] == t) & (df_t1_3y['FTR'] =='H'))]))
#             d3["t1_w3yr"].append((len(df_t1_3y[((df_t1_3y['HomeTeam'] == t) & (df_t1_3y['FTR'] =='H')) | ((df_t1_3y['AwayTeam'] == t) & (df_t1_3y['FTR'] =='A'))]))/len(df_t1_3y))
#             d3["t1_g3yr"].append((len(df_t1_3y[((df_t1_3y['HomeTeam'] == t) & (df_t1_3y['FTR'] =='D')) | ((df_t1_3y['AwayTeam'] == t) & (df_t1_3y['FTR'] =='D'))]))/len(df_t1_3y))
#             d3["t1_v3yr"].append((len(df_t1_3y[((df_t1_3y['HomeTeam'] == t) & (df_t1_3y['FTR'] =='A')) | ((df_t1_3y['AwayTeam'] == t) & (df_t1_3y['FTR'] =='H'))]))/len(df_t1_3y))
#             d3['t1_tg3y'].append((sum(df_t1_3y['FTHG']) + sum(df_t1_3y['FTAG']))/len(df_t1_3y))
#             d3['t1_gf3y'].append(((sum(df_t1_3y['FTHG'][((df_t1_3y['HomeTeam'] == t))])+(sum(df_t1_3y['FTAG'][((df_t1_3y['AwayTeam'] == t))])))/len(df_t1_3y)))
#             d3['t1_ga3y'].append(((sum(df_t1_3y['FTHG'][((df_t1_3y['HomeTeam'] != t))])+(sum(df_t1_3y['FTAG'][((df_t1_3y['AwayTeam'] != t))])))/len(df_t1_3y)))
#         else:
#             d3["t1_w3y"].append(np.nan)
#             d3["t1_g3y"].append(np.nan)
#             d3["t1_v3y"].append(np.nan)
#             d3["t1_w3yr"].append(np.nan)
#             d3["t1_g3yr"].append(np.nan)
#             d3["t1_v3yr"].append(np.nan)
#             d3['t1_tg3y'].append(np.nan)
#             d3['t1_gf3y'].append(np.nan)

#         if len(df_t1_tt_3y) > 0:
#             d3["t1_tt_w3y"].append(len(df_t1_tt_3y[((df_t1_tt_3y['HomeTeam'] == t) & (df_t1_tt_3y['FTR'] =='H'))]))
#             d3["t1_tt_g3y"].append(len(df_t1_tt_3y[((df_t1_tt_3y['HomeTeam'] == t) & (df_t1_tt_3y['FTR'] =='D'))]))
#             d3["t1_tt_v3y"].append(len(df_t1_tt_3y[((df_t1_tt_3y['HomeTeam'] == t) & (df_t1_tt_3y['FTR'] =='A'))]))
#             d3["t1_tt_w3yr"].append((len(df_t1_tt_3y[((df_t1_tt_3y['HomeTeam'] == t) & (df_t1_tt_3y['FTR'] =='H'))]))/len(df_t1_tt_3y))
#             d3["t1_tt_g3yr"].append((len(df_t1_tt_3y[((df_t1_tt_3y['HomeTeam'] == t) & (df_t1_tt_3y['FTR'] =='D'))]))/len(df_t1_tt_3y))
#             d3["t1_tt_v3yr"].append((len(df_t1_tt_3y[((df_t1_tt_3y['HomeTeam'] == t) & (df_t1_tt_3y['FTR'] =='A'))]))/len(df_t1_tt_3y))

#             d3['t1_tt_tg3y'].append((sum(df_t1_tt_3y['FTHG']) + sum(df_t1_tt_3y['FTAG']))/len(df_t1_tt_3y))
#             d3['t1_tt_gf3y'].append((sum(df_t1_tt_3y['FTHG'][((df_t1_tt_3y['HomeTeam'] == t))])/len(df_t1_tt_3y)))
#             d3['t1_tt_ga3y'].append((sum(df_t1_tt_3y['FTAG'][((df_t1_tt_3y['HomeTeam'] == t))])/len(df_t1_tt_3y)))
#         else:
#             d3["t1_tt_w3y"].append(np.nan)
#             d3["t1_tt_g3y"].append(np.nan)
#             d3["t1_tt_v3y"].append(np.nan)
#             d3["t1_tt_w3yr"].append(np.nan)
#             d3["t1_tt_g3yr"].append(np.nan)
#             d3["t1_tt_v3yr"].append(np.nan)
#             d3['t1_tt_tg3y'].append(np.nan)
#             d3['t1_tt_gf3y'].append(np.nan)
#             d3['t1_tt_ga3y'].append(np.nan)

#         if len(df_t1_ut_3y) != 0:
#             d3["t1_ut_w3y"].append(len(df_t1_ut_3y[((df_t1_ut_3y['AwayTeam'] == t) & (df_t1_ut_3y['FTR'] =='H'))]))
#             d3["t1_ut_g3y"].append(len(df_t1_ut_3y[((df_t1_ut_3y['AwayTeam'] == t) & (df_t1_ut_3y['FTR'] =='D'))]))
#             d3["t1_ut_v3y"].append(len(df_t1_ut_3y[((df_t1_ut_3y['AwayTeam'] == t) & (df_t1_ut_3y['FTR'] =='A'))]))
#             d3["t1_ut_w3yr"].append((len(df_t1_ut_3y[((df_t1_ut_3y['AwayTeam'] == t) & (df_t1_ut_3y['FTR'] =='H'))]))/len(df_t1_ut_3y))
#             d3["t1_ut_g3yr"].append((len(df_t1_ut_3y[((df_t1_ut_3y['AwayTeam'] == t) & (df_t1_ut_3y['FTR'] =='D'))]))/len(df_t1_ut_3y))
#             d3["t1_ut_v3yr"].append((len(df_t1_ut_3y[((df_t1_ut_3y['AwayTeam'] == t) & (df_t1_ut_3y['FTR'] =='A'))]))/len(df_t1_ut_3y))

#             d3['t1_ut_tg3y'].append((sum(df_t1_ut_3y['FTHG']) + sum(df_t1_ut_3y['FTAG']))/len(df_t1_ut_3y))
#             d3['t1_ut_gf3y'].append((sum(df_t1_ut_3y['FTHG'][((df_t1_ut_3y['AwayTeam'] == t))])/len(df_t1_ut_3y)))
#             d3['t1_ut_ga3y'].append((sum(df_t1_ut_3y['FTAG'][((df_t1_ut_3y['AwayTeam'] == t))])/len(df_t1_ut_3y)))
#         else:
#             d3["t1_ut_w3y"].append(np.nan)
#             d3["t1_ut_g3y"].append(np.nan)
#             d3["t1_ut_v3y"].append(np.nan)
#             d3["t1_ut_w3yr"].append(np.nan)
#             d3["t1_ut_g3yr"].append(np.nan)
#             d3["t1_ut_v3yr"].append(np.nan)
#             d3['t1_ut_tg3y'].append(np.nan)
#             d3['t1_ut_gf3y'].append(np.nan)
#             d3['t1_ut_ga3y'].append(np.nan)

#         if len(df_t2_3y) != 0:
#             d3["t2_w3y"].append(len(df_t2_3y[((df_t2_3y['HomeTeam'] == u) & (df_t2_3y['FTR'] =='H')) | ((df_t2_3y['AwayTeam'] == u) & (df_t2_3y['FTR'] =='A'))]))
#             d3["t2_g3y"].append((len(df_t2_3y[((df_t2_3y['HomeTeam'] == u) & (df_t2_3y['FTR'] =='D')) | ((df_t2_3y['AwayTeam'] == u) & (df_t2_3y['FTR'] =='D'))])))
#             d3["t2_v3y"].append(len(df_t2_3y[((df_t2_3y['HomeTeam'] == u) & (df_t2_3y['FTR'] =='A')) | ((df_t2_3y['AwayTeam'] == u) & (df_t2_3y['FTR'] =='H'))]))
#             d3["t2_w3yr"].append((len(df_t2_3y[((df_t2_3y['HomeTeam'] == u) & (df_t2_3y['FTR'] =='H')) | ((df_t2_3y['AwayTeam'] == u) & (df_t2_3y['FTR'] =='A'))]))/len(df_t2_3y))
#             d3["t2_g3yr"].append((len(df_t2_3y[((df_t2_3y['HomeTeam'] == u) & (df_t2_3y['FTR'] =='D')) | ((df_t2_3y['AwayTeam'] == u) & (df_t2_3y['FTR'] =='D'))]))/len(df_t2_3y))
#             d3["t2_v3yr"].append((len(df_t2_3y[((df_t2_3y['HomeTeam'] == u) & (df_t2_3y['FTR'] =='A')) | ((df_t2_3y['AwayTeam'] == u) & (df_t2_3y['FTR'] =='H'))]))/len(df_t2_3y))
#             d3['t2_tg3y'].append((sum(df_t2_3y['FTHG']) + sum(df_t2_3y['FTAG']))/len(df_t2_3y))
#             d3['t2_gf3y'].append(((sum(df_t2_3y['FTHG'][((df_t2_3y['HomeTeam'] == u))])+(sum(df_t2_3y['FTAG'][((df_t2_3y['AwayTeam'] == t))])))/len(df_t2_3y)))
#             d3['t2_ga3y'].append(((sum(df_t2_3y['FTHG'][((df_t2_3y['HomeTeam'] != u))])+(sum(df_t2_3y['FTAG'][((df_t2_3y['AwayTeam'] != t))])))/len(df_t2_3y)))
#         else:
#             d3["t2_w3y"].append(np.nan)
#             d3["t2_g3y"].append(np.nan)
#             d3["t2_v3y"].append(np.nan)
#             d3["t2_w3yr"].append(np.nan)
#             d3["t2_g3yr"].append(np.nan)
#             d3["t2_v3yr"].append(np.nan)
#             d3['t2_tg3y'].append(np.nan)
#             d3['t2_gf3y'].append(np.nan)

#         if len(df_t2_tt_3y) != 0:
#             d3["t2_tt_w3y"].append(len(df_t2_tt_3y[((df_t2_tt_3y['HomeTeam'] == u) & (df_t2_tt_3y['FTR'] =='H'))]))
#             d3["t2_tt_g3y"].append(len(df_t2_tt_3y[((df_t2_tt_3y['HomeTeam'] == u) & (df_t2_tt_3y['FTR'] =='D'))]))
#             d3["t2_tt_v3y"].append(len(df_t2_tt_3y[((df_t2_tt_3y['HomeTeam'] == u) & (df_t2_tt_3y['FTR'] =='A'))]))
#             d3["t2_tt_w3yr"].append((len(df_t2_tt_3y[((df_t2_tt_3y['HomeTeam'] == u) & (df_t2_tt_3y['FTR'] =='H'))]))/len(df_t2_tt_3y))
#             d3["t2_tt_g3yr"].append((len(df_t2_tt_3y[((df_t2_tt_3y['HomeTeam'] == u) & (df_t2_tt_3y['FTR'] =='D'))]))/len(df_t2_tt_3y))
#             d3["t2_tt_v3yr"].append((len(df_t2_tt_3y[((df_t2_tt_3y['HomeTeam'] == u) & (df_t2_tt_3y['FTR'] =='A'))]))/len(df_t2_tt_3y))

#             d3['t2_tt_tg3y'].append((sum(df_t2_tt_3y['FTHG']) + sum(df_t2_tt_3y['FTAG']))/len(df_t2_tt_3y))
#             d3['t2_tt_gf3y'].append((sum(df_t2_tt_3y['FTHG'][((df_t2_tt_3y['HomeTeam'] == u))])/len(df_t2_tt_3y)))
#             d3['t2_tt_ga3y'].append((sum(df_t2_tt_3y['FTAG'][((df_t2_tt_3y['HomeTeam'] == u))])/len(df_t2_tt_3y)))
#         else:
#             d3["t2_tt_w3y"].append(np.nan)
#             d3["t2_tt_g3y"].append(np.nan)
#             d3["t2_tt_v3y"].append(np.nan)
#             d3["t2_tt_w3yr"].append(np.nan)
#             d3["t2_tt_g3yr"].append(np.nan)
#             d3["t2_tt_v3yr"].append(np.nan)
#             d3['t2_tt_tg3y'].append(np.nan)
#             d3['t2_tt_gf3y'].append(np.nan)
#             d3['t2_tt_ga3y'].append(np.nan)

#         if len(df_t2_ut_3y) != 0:
#             d3["t2_ut_w3y"].append(len(df_t2_ut_3y[((df_t2_ut_3y['AwayTeam'] == u) & (df_t2_ut_3y['FTR'] =='H'))]))
#             d3["t2_ut_g3y"].append(len(df_t2_ut_3y[((df_t2_ut_3y['AwayTeam'] == u) & (df_t2_ut_3y['FTR'] =='D'))]))
#             d3["t2_ut_v3y"].append(len(df_t2_ut_3y[((df_t2_ut_3y['AwayTeam'] == u) & (df_t2_ut_3y['FTR'] =='A'))]))
#             d3["t2_ut_w3yr"].append((len(df_t2_ut_3y[((df_t2_ut_3y['AwayTeam'] == u) & (df_t2_ut_3y['FTR'] =='H'))]))/len(df_t2_ut_3y))
#             d3["t2_ut_g3yr"].append((len(df_t2_ut_3y[((df_t2_ut_3y['AwayTeam'] == u) & (df_t2_ut_3y['FTR'] =='D'))]))/len(df_t2_ut_3y))
#             d3["t2_ut_v3yr"].append((len(df_t2_ut_3y[((df_t2_ut_3y['AwayTeam'] == u) & (df_t2_ut_3y['FTR'] =='A'))]))/len(df_t2_ut_3y))

#             d3['t2_ut_tg3y'].append((sum(df_t2_ut_3y['FTHG']) + sum(df_t2_ut_3y['FTAG']))/len(df_t2_ut_3y))
#             d3['t2_ut_gf3y'].append((sum(df_t2_ut_3y['FTAG'][((df_t2_ut_3y['AwayTeam'] == u))])/len(df_t2_ut_3y)))
#             d3['t2_ut_ga3y'].append((sum(df_t2_ut_3y['FTHG'][((df_t2_ut_3y['AwayTeam'] == u))])/len(df_t2_ut_3y)))
#         else:
#             d3["t2_ut_w3y"].append(np.nan)
#             d3["t2_ut_g3y"].append(np.nan)
#             d3["t2_ut_v3y"].append(np.nan)
#             d3["t2_ut_w3yr"].append(np.nan)
#             d3["t2_ut_g3yr"].append(np.nan)
#             d3["t2_ut_v3yr"].append(np.nan)
#             d3['t2_ut_tg3y'].append(np.nan)
#             d3['t2_ut_gf3y'].append(np.nan)
#             d3['t2_ut_ga3y'].append(np.nan)

